In [1]:
require("Rsamtools")
require("Hmisc")
require("dplyr")
require("ggplot2")
require("stringi")
require("stats")
require("Biostrings")
require("data.table")
require("tidyverse")
require("Rtsne")
require("patchwork")
require("R.utils")
require("pheatmap")
require("ggridges")
require("TSdist")
require("dtplyr")
require("profvis")
require("stringr")
require("Biostrings")
require("ggforce")
require("pracma")
require("rstatix")
require("ggpubr")
require("jsonlite")
require("matrixStats")

Loading required package: Rsamtools

Loading required package: GenomeInfoDb

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    Position, rank, rbind, Reduce, rownames, sapply, setdiff, sort,
    table, tapply, union, unique, unsplit, which.max, which.min


Loading required package: S4Vectors

Loading required package: stats4


Attaching package: ‘S4Vectors’


The following objects are masked from ‘package:base’:

    expand.grid, I, unname


Loading required package: IRanges

Loading required package: GenomicRanges

Loading required package: Biostrings



Warning message:
“no DISPLAY variable so Tk is not available”
Registered S3 method overwritten by 'quantmod':
  method            from
  as.zoo.data.frame zoo 

Loaded TSdist v3.7.1. See ?TSdist for help, citation("TSdist") for use in publication.


Loading required package: dtplyr

Loading required package: profvis

Loading required package: ggforce

Loading required package: pracma


Attaching package: ‘pracma’


The following object is masked from ‘package:purrr’:

    cross


The following object is masked from ‘package:Hmisc’:

    ceil


Loading required package: rstatix


Attaching package: ‘rstatix’


The following object is masked from ‘package:IRanges’:

    desc


The following object is masked from ‘package:stats’:

    filter


Loading required package: ggpubr

Loading required package: jsonlite


Attaching package: ‘jsonlite’


The following object is masked from ‘package:R.utils’:

    validate


The following object is masked from ‘package:purrr’:

    flatten


Loading

In [4]:
#function loading of fastq file, derived from https://github.com/cran/microseq/blob/master/R/fastq.R
readFastq <- function(in.file){
  if(!is.character(in.file) | length(in.file)>1) stop("The argument in.file must be a single text (a filename)")
  if(file.exists(in.file)){
    in.file <- normalizePath(in.file)
    tbl <- fread(in.file, header = F, sep = "\t", data.table = F, quote = "")
    tibble(Header   = str_remove(tbl[seq(1, nrow(tbl), 4),1], "^@"),
           Sequence = tbl[seq(2, nrow(tbl), 4),1],
           Quality  = tbl[seq(4, nrow(tbl), 4),1]) -> fdta
    return(fdta)
  } else {
    stop("Cannot find ", in.file, ", please correct path and/or file name")
  }
}

In [ ]:
# Function to calculate the index for each valley
calculate_index_for_valley <- function(valley_TLEN, valley_mnsCount, peaks) {
    # Find peaks within the range of less than 8 absolute difference
    nearby_peaks <- peaks %>%
        filter(abs(TLEN - valley_TLEN) <= 9)

    # If there are exactly 2 peaks, calculate the index
    if (nrow(nearby_peaks) == 2) {
        mean_peak_mnsCount <- mean(nearby_peaks$mnsCount)
        index <- mean_peak_mnsCount / valley_mnsCount
        return(index)
    } else {
        return(NA)  # Return NA if the condition isn't met
    }
}

# Function to get indices for all valleys in a sample type
get_index_for_valley <- function(host_nonMT_summarized, sample_type_tmp, tlen_min = 150) {
    subsel_df <- host_nonMT_summarized %>% filter(sample_type == sample_type_tmp)
    
    # Identify peaks
    peaks_subsel_df <- findpeaks(subsel_df$mnsCount, minpeakdistance = 7, npeaks = 25, threshold = 0)
    df_peaks <- subsel_df[c(peaks_subsel_df[, 2]), ] %>% mutate(type = "peak")
    
    # Identify valleys (invert signal to find valleys)
    valleys_subsel_df <- findpeaks(-subsel_df$mnsCount, minpeakdistance = 7, npeaks = 25, threshold = 0)
    df_valleys <- subsel_df[c(valleys_subsel_df[, 2]), ] %>% mutate(type = "valley")
    
    peaks_valleys <- rbind(df_peaks, df_valleys) %>% filter(TLEN < tlen_min)
    
    # Calculate index for each valley
    valley_index <- df_valleys %>%
        mutate(index = map2_dbl(TLEN, mnsCount, ~ calculate_index_for_valley(.x, .y, df_peaks))) %>% 
        filter(TLEN < tlen_min) %>% 
        group_by(sample_type) %>% 
        summarise(
            mean_index = round(mean(index, na.rm = TRUE), digits = 2),
            
            index_non_na = sum(!is.na(index)), 
            index_na = sum(is.na(index)), 
            total_index = sum(index_non_na, index_na)
    )

    # Return the results
    return(list(peaks_valleys, valley_index))
}